# NER модель 
# для удаления названий компаний из текстов

Названия компаний, в основном, это идентификаторы на которых переобучается модель. Разработаем автоматический фильтр названий компаний для предобработки набора данных.

Модель основана на эвристрических правилах обработки текстовых строк.

In [19]:
import re

class NERFilter:
    def __init__(self):
        self._lowFreqWords = []

    # Удаляем подстроки с большой буквы внутри кавычек «»
    def _ruleDoubleAngleQuotation(self, s):
        result = s
        pattern = '[ (]«[А-ЯЁA-Z][^«»]*»'
        matches = re.findall(pattern, s)

        for m in matches:
            result = result.replace(m, '')

        return result
    
    # Удаляем низкочастотные подстроки с большой буквы
    def _ruleLowFreqsWithCaps(self, s):
        result = []
        for w in s.split(' '):
            if w not in self._lowFreqWords:
                result.append(w)

        return ' '.join(result)
    
     
    def fit(self, corpus, importantWords, level = 10):
        """
        Args:
            corpus — список строк;
            importantWords — список слов для сохранения (такие слова не будут удаляться);
            level — уровень кол-ва появления слова в текстах, чтобы его отфильтровать 
                    (слова с большой буквы с частотностью ниже level будут удалены
                    при вызове метода filter).
        """
        freqs = {}

        # Get frequencies of words:
        for txt in corpus:
            for w in txt.split(' '):
                #print('"'+w+'"')
                if len(w) > 1 \
                        and w[0].upper() == w[0] \
                        and w.lower() not in importantWords \
                        and '-' not in w:
                    if w in freqs:
                        freqs[w] = freqs[w] + 1
                    else:
                        freqs[w] = 1

        # Write results:
        
        for k, v in freqs.items():
            if v < level:
                self._lowFreqWords.append(k)

        return self._lowFreqWords
    

    def filter(self, s):
        """
        Метод для фильтрации названий из текста.
        """
        result = self._ruleDoubleAngleQuotation(s)
        result = self._ruleLowFreqsWithCaps(result)
        return result.replace('  ', ' ')

In [20]:
txt = "IVAT it  IVA Technologies - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций IVA.  В экосистему IVA входят: корпоративный мессенджер IVA Connect платформа видеоконференцсвязи IVA MCU облачная платформа видеоконференцсвязи «ВКурсе» помощник основанный на технологии искусственного интеллекта IVA GPT система управления и мониторинга IVA MS сервер корпоративной телефонии IVA CS серия IP-телефонов видеотерминалы IVA Room и IVA Largo пограничный контроллер сессий IVA SBC и другие продукты."
corpus = [txt,]
importantWords = ['gpt',]

nerModel = NERFilter()
nerModel.fit(corpus, importantWords, level = 15)
nerModel.filter(txt)


'it - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций В экосистему входят: корпоративный мессенджер платформа видеоконференцсвязи облачная платформа видеоконференцсвязи помощник основанный на технологии искусственного интеллекта GPT система управления и мониторинга сервер корпоративной телефонии серия IP-телефонов видеотерминалы и пограничный контроллер сессий и другие продукты.'

Мы получили NER модель, которую можно использовать для предобработки текстов описаний компаний в развитом ПО, которое будет использовать ML модель.